# Python AI Chess Game


## Importing Libraries

In [ ]:
import chess
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import clear_output
from collections import OrderedDict
from operator import itemgetter

## Functions

In [ ]:
def predict(df, model):
    col_names = df.columns
    dtypes = df.dtypes
    predictions = []
    for row in df.iterrows():
        example = tf.train.Example()
        for i in range(len(col_names)):
            dtype = dtypes[i]
            col_name = col_names[i]
            value = row[1][col_name]
            if dtype == 'object':
                value = bytes(value, 'utf-8')
                example.features.feature[col_name].bytes_list.value.extend([value])
            elif dtype == 'float':
                example.features.feature[col_name].float_list.value.extend([value])
            elif dtype == 'int':
                example.features.feature[col_name].int64_list.value.extend([value])
        predictions.append(model.signatures['predict'](examples = tf.constant([example.SerializeToString()])))
    return predictions

# Function to create a dataframe used to make predictions
def createMovesDataframe(board):
    squareFeatures = []
    for i in chess.SQUARES:
        squareFeatures.append(str(board.piece_at(i)))
        
    data = []
    moves = list(board.legal_moves)
    for i in moves:
        a = np.zeros(64)
        b = np.zeros(64)
        a[i.from_square] = 1
        b[i.to_square] = 1
        fromSquare = a
        toSquare = b
        row = np.concatenate((squareFeatures, fromSquare, toSquare))
        data.append(row)
    
    squareNames = chess.SQUARE_NAMES
    moveFromFeatures = ['from_' + i for i in chess.SQUARE_NAMES]
    moveToFeatures = ['to_' + i for i in chess.SQUARE_NAMES]
    
    columns = squareNames + moveFromFeatures + moveToFeatures
    
    df = pd.DataFrame(data = data, columns = columns)

    for i in moveFromFeatures:
        df[i] = df[i].astype(float)
    for i in moveToFeatures:
        df[i] = df[i].astype(float)
    return df

# Function to get the value of a piece on the chess board
def getPieceValue(piece, square):
    
    x, y = {0:(7,0), 1:(7,1), 2:(7,2), 3:(7,3), 4:(7,4), 5:(7,5), 6:(7,6), 7:(7,7),
          8:(6,0), 9:(6,1), 10:(6,2), 11:(6,3), 12:(6,4), 13:(6,5), 14:(6,6), 15:(6,7), 
          16:(5,0), 17:(5,1), 18:(5,2), 19:(5,3), 20:(5,4), 21:(5,5), 22:(5,6), 23:(5,7),
          24:(4,0), 25:(4,1), 26:(4,2), 27:(4,3), 28:(4,4), 29:(4,5), 30:(4,6), 31:(4,7),
          32:(3,0), 33:(3,1), 34:(3,2), 35:(3,3), 36:(3,4), 37:(3,5), 38:(3,6), 39:(3,7),
          40:(2,0), 41:(2,1), 42:(2,2), 43:(2,3), 44:(2,4), 45:(2,5), 46:(2,6), 47:(2,7),
          48:(1,0), 49:(1,1), 50:(1,2), 51:(1,3), 52:(1,4), 53:(1,5), 54:(1,6), 55:(1,7),
          56:(0,0), 57:(0,1), 58:(0,2), 59:(0,3), 60:(0,4), 61:(0,5), 62:(0,6), 63:(0,7)}[square]
    
    if(AI_black):
        white = -1
        black = 1
    else:
        white = 1
        black = -1

    if(piece == 'None'):
        return 0
    elif(piece == 'P'):
        return white * (10 + pawn_white_eval[x][y])
    elif(piece == 'N'):
        return white * (30 + knight_white_eval[x][y])
    elif(piece == 'B'):
        return white * (30 + bishop_white_eval[x][y])
    elif(piece == 'R'):
        return white * (50 + rook_white_eval[x][y])
    elif(piece == 'Q'):
        return white * (90 + queen_white_eval[x][y])
    elif(piece == 'K'):
        return white * (900 + king_white_eval[x][y])
    elif(piece == 'p'):
        return black * (10 + pawn_black_eval[x][y])
    elif(piece == 'n'):
        return black * (30 + knight_black_eval[x][y])
    elif(piece == 'b'):
        return black * (30 + bishop_black_eval[x][y])
    elif(piece == 'r'):
        return black * (50 + rook_black_eval[x][y])
    elif(piece == 'q'):
        return black * (90 + queen_black_eval[x][y])
    elif(piece == 'k'):
        return black * (900 + king_black_eval[x][y])

# Auxiliar function to implement minimax algorithm
def minimax_aux(depth, board, alpha, beta, guard):
    if(depth == 0):
        boardEvaluation = 0
        for i in chess.SQUARES:
            piece = str(board.piece_at(i))
            boardEvaluation = boardEvaluation + getPieceValue(piece, i)
        return - boardEvaluation
    
    elif(depth > 3):
        movesList = list(board.legal_moves)
        df = createMovesDataframe(board)
        predictions = predict(df, model)
        goodMoveProbabilities = []   
        for i in predictions:
            proto_tensor = tf.make_tensor_proto(i['probabilities'])
            probabilities = tf.make_ndarray(proto_tensor)[0][1]
            goodMoveProbabilities.append(probabilities)
    
        dictionar = dict(zip(movesList, goodMoveProbabilities))
        dictionar = OrderedDict(sorted(dictionar.items(), key = itemgetter(1), reverse = True))
    
        bestMoves = list(dictionar.keys())    
        legalMoves = bestMoves[0:int(len(bestMoves)*0.75)]
    else:
        legalMoves = list(board.legal_moves)

    if(guard):
        bestMove = -9999
        for i in legalMoves:
            board.push(i)
            bestMove = max(bestMove, minimax_aux(depth-1, board, alpha, beta, not guard))
            board.pop()
            alpha = max(alpha, bestMove)
            if(beta <= alpha):
                return bestMove
        return bestMove
    else:
        bestMove = 9999
        for i in legalMoves:
            board.push(i)
            bestMove = min(bestMove, minimax_aux(depth-1, board, alpha, beta, not guard))
            board.pop()
            beta = min(beta, bestMove)
            if(beta <= alpha):
                return bestMove
        return bestMove

# Function to implement minimax algorithm
def minimax(depth, board):
    movesList = list(board.legal_moves)
    df = createMovesDataframe(board)
    predictions = predict(df, model)
    goodMoveProbabilities = []   
    for i in predictions:
        proto_tensor = tf.make_tensor_proto(i['probabilities'])
        probabilities = tf.make_ndarray(proto_tensor)[0][1]
        goodMoveProbabilities.append(probabilities)
    
    dictionar = dict(zip(movesList, goodMoveProbabilities))
    dictionar = OrderedDict(sorted(dictionar.items(), key = itemgetter(1), reverse = True))
    
    bestMoves = list(dictionar.keys())    
    legalMoves = bestMoves[0:int(len(bestMoves)*0.75)]
    bestMove = -9999
    bestMoveCalculated = None
    guard = True
    
    for i in legalMoves:
        board.push(i)
        value = minimax_aux(depth - 1, board, -10000, 10000, not guard)
        board.pop()
        if(value >= bestMove):
            bestMove = value
            bestMoveCalculated = i

    return bestMoveCalculated

# Function to implement the checkmate
def checkmate(move, board):
    FEN = board.fen()
    boardCheckMate = chess.Board(FEN)
    boardCheckMate.push(move)
    return boardCheckMate.is_checkmate()

# Function to start the game
def startGame(guard, board):
    
    if(board.is_stalemate()):
        clear_output()
        print("Stalemate!")
        return
    else:   
        if(guard):
            if(not board.is_checkmate()):
                clear_output()
                display(board)
                inputMove = input('Enter your move: ')
                move = chess.Move.from_uci(inputMove)
                if(move not in board.legal_moves):
                    return startGame(guard, board)
    
                board.push(move)
                return startGame(not guard, board)
            else:
                clear_output()
                display(board)
                print('AI wins!')
                return
        else:
            if(not board.is_checkmate()):
                clear_output()
                display(board)
                for move in board.legal_moves:
                    if(checkmate(move, board)):
                        board.push(move)
                        return
                
                nb_moves = len(list(board.legal_moves))
   
                if(nb_moves > 30):
                    board.push(minimax(4, board))
                elif(nb_moves > 10 and nb_moves <= 30):
                    board.push(minimax(5, board))
                else:
                    board.push(minimax(7, board))
             
                return startGame(not guard, board)
            
            else:
                clear_output()
                display(board)
                print("User wins!")
                return

## Pieces Initialization Values

In [ ]:
pawn_white_eval = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                            [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0],
                            [1.0, 1.0, 2.0, 3.0, 3.0, 2.0, 1.0, 1.0],
                            [0.5, 0.5, 1.0, 2.5, 2.5, 1.0, 0.5, 0.5],
                            [0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0],
                            [0.5, -0.5, -1.0, 0.0, 0.0, -1.0, -0.5, 0.5],
                            [0.5, 1.0, 1.0, -2.0, -2.0, 1.0, 1.0, 0.5],
                            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], np.double)

pawn_black_eval = pawn_white_eval[::-1]

knight_white_eval = np.array([[-5.0, -4.0, -3.0, -3.0, -3.0, -3.0, -4.0, -5.0],
                              [-4.0, -2.0, 0.0, 0.0, 0.0, 0.0, -2.0, -4.0],
                              [-3.0, 0.0, 1.0, 1.5, 1.5, 1.0, 0.0, -3.0],
                              [-3.0, 0.5, 1.5, 2.0, 2.0, 1.5, 0.5, -3.0],
                              [-3.0, 0.0, 1.5, 2.0, 2.0, 1.5, 0.0, -3.0],
                              [-3.0, 0.5, 1.0, 1.5, 1.5, 1.0, 0.5, -3.0],
                              [-4.0, -2.0, 0.0, 0.5, 0.5, 0.0, -2.0, -4.0],
                              [-5.0, -4.0, -3.0, -3.0, -3.0, -3.0, -4.0, -5.0]], np.double)

knight_black_eval = knight_white_eval[::-1]

bishop_white_eval = np.array([[-2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -2.0],
                              [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0],
                              [-1.0, 0.0, 0.5, 1.0, 1.0, 0.5, 0.0, -1.0],
                              [-1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, -1.0],
                              [-1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, -1.0],
                              [-1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0],
                              [-1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, -1.0],
                              [-2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -2.0]], np.double)

bishop_black_eval = bishop_white_eval[::-1]

rook_white_eval = np.array([[0.0, 0.0, 0.0, 0.0, 0.0,  0.0, 0.0, 0.0],
                            [0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [ 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0]], np.double)

rook_black_eval = rook_white_eval[::-1]

queen_white_eval = np.array([[-2.0, -1.0, -1.0, -0.5, -0.5, -1.0, -1.0, -2.0],
                             [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0],
                             [-1.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, -1.0],
                             [-0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, -0.5],
                             [0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, -0.5],
                             [-1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, -1.0],
                             [-1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, -1.0],
                             [-2.0, -1.0, -1.0, -0.5, -0.5, -1.0, -1.0, -2.0]], np.double)

queen_black_eval = queen_white_eval[::-1]

king_white_eval = np.array([[-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
                            [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
                            [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
                            [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
                            [-2.0, -3.0, -3.0, -4.0, -4.0, -3.0, -3.0, -2.0],
                            [-1.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -1.0],
                            [2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0],
                            [2.0, 3.0, 1.0, 0.0, 0.0, 1.0, 3.0, 2.0]], np.double)

king_black_eval = king_white_eval[::-1]

## Upload the AI model simulating Bobby Fischer

In [ ]:
global model
model = tf.saved_model.load("Model")

## Play vs Bobby Fischer

In [ ]:
global AI_black
board = chess.Board()
print("Welcome to AI Chess! You are going to play vs Bobby Fischer")
print("To enter a move, write in lowercase the starting cell of the chess board followed by the destination cell. Example: b1c3")
user_starting = input("Would you start? [y/n]: ")
if(user_starting == 'y'):
    AI_black = True
    startGame(True, board)
else:
    AI_black = False
    startGame(False, board)